In [36]:

import tensorflow as tf
import keras
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
import numpy as np
import cv2
import os
import time



1.14.0


In [29]:



# BUILDING MODEL TO CLASSIFY BETWEEN MASK AND NO MASK

model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
training_set = train_datagen.flow_from_directory(
        'C:\\Users\\adith\\Desktop\\FaceMaskDetector-master\\train',
        target_size=(150,150),
        batch_size=16 ,
        class_mode='binary')
test_set = test_datagen.flow_from_directory(
        'C:\\Users\\adith\Desktop\\FaceMaskDetector-master\\test',
        target_size=(150,150),
        batch_size=16,
        class_mode='binary')
model_saved=model.fit_generator(
        training_set,
        epochs=10,
        validation_data=test_set,
        )
model.save('mymodel.h5',model_saved)


Found 1314 images belonging to 2 classes.
Found 194 images belonging to 2 classes.
Epoch 1/10
83/83 [==============================] - 23s 282ms/step - loss: 0.3927 - accuracy: 0.8288 - val_loss: 1.0554 - val_accuracy: 0.9381
Epoch 2/10
83/83 [==============================] - 23s 283ms/step - loss: 0.1491 - accuracy: 0.9437 - val_loss: 0.0096 - val_accuracy: 0.9072
Epoch 3/10
83/83 [==============================] - 24s 284ms/step - loss: 0.1083 - accuracy: 0.9635 - val_loss: 0.0041 - val_accuracy: 0.9639
Epoch 4/10
83/83 [==============================] - 25s 299ms/step - loss: 0.0870 - accuracy: 0.9696 - val_loss: 1.3362e-04 - val_accuracy: 0.9588
Epoch 5/10
83/83 [==============================] - 25s 301ms/step - loss: 0.0805 - accuracy: 0.9734 - val_loss: 0.0172 - val_accuracy: 0.9897
Epoch 6/10
83/83 [==============================] - 24s 294ms/step - loss: 0.0666 - accuracy: 0.9764 - val_loss: 0.0019 - val_accuracy: 0.9742
Epoch 7/10
83/83 [==============================] - 24s

In [57]:
mymodel=load_model('mymodel.h5')

cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier('C:\\Users\\adith\\Anaconda3\\Library\\etc\\haarcascades\\haarcascade_frontalface_default.xml')

# FOR REAL TIME DETECTION

while cap.isOpened():
    _,img=cap.read()
    face=face_cascade.detectMultiScale(img,scaleFactor=1.1,minNeighbors=4)
    for(x,y,w,h) in face:
        face_img = img[y:y+h, x:x+w]
        cv2.imwrite('temp.jpg',face_img)
        test_image=image.load_img('temp.jpg',target_size=(150,150,3))
        test_image=image.img_to_array(test_image)
        test_image=np.expand_dims(test_image,axis=0)
        pred=mymodel.predict_classes(test_image)[0][0]
        if pred==1:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)
            cv2.putText(img,'NO MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)
        else:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),3)
            cv2.putText(img,'MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),3)
        datet=str(datetime.datetime.now())
        cv2.putText(img,datet,(400,450),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)
          
    cv2.imshow('img',img)
    
    if cv2.waitKey(1)==ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()
